In [4]:
# -*- coding: UTF-8 -*-

import sys
import os
import collections
import re
import numpy as np
import matplotlib.pyplot as plt
import torch

# 马尔可夫链（Markov chain of order n）
with open('E:\\code\\python_workSpace\\idea_space\\Deep_Learning\\1_Programming_Framework\\2_Pytorch\\2_LiMu_Pytorch_MyLearn\\data\\jaychou_lyrics.txt', encoding='utf-8') as f:
    corpus_chars = f.read()
print(len(corpus_chars))
print(corpus_chars[: 40])
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[: 10000]

63282
想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每


In [8]:
# 建立字符索引
idx_to_char = list(set(corpus_chars)) # 去重，得到索引到字符的映射
char_to_idx = {char:i for i, char in enumerate(idx_to_char)} # 字符到索引的映射
vocab_size = len(char_to_idx)
print(vocab_size)
# print(idx_to_char)
# print(char_to_idx)

1027
['证', '任', '塞', '纪', '惯', '爵', '深', '滩', '世', '瓜', '玫', '地', '可', '蜜', '囱', '啦', '替', '前', '撑', '哭', '分', '往', '转', '蛦', '币', '彷', '悄', '害', '全', '骑', '拽', '威', '留', '吹', '湖', '衷', '相', '榜', '整', '题', '诉', '它', '钢', '栈', '肉', '你', '护', '油', '望', '荡', '该', '路', '比', '腿', '迎', '舍', '苦', '嘟', '童', '双', '还', '坟', '抱', '祈', '果', '临', '哀', '怎', '丁', '蓝', '明', '罩', '以', '右', '行', '封', '无', '擅', '雅', '雨', '切', '中', '散', '之', '斗', '非', '产', '干', '秀', '方', '小', '胖', '歌', '喜', '暴', '横', '亏', '铜', '别', '秋', '辈', '呼', '语', '片', '帅', '里', '巫', '夕', '送', '法', '饿', '水', '操', '祷', '药', '司', '酒', '后', '再', '外', '格', '霓', '酱', '错', '铁', '黑', '区', '征', '楷', '拆', '或', '兵', '奇', '爹', '简', '泽', '指', '景', '芜', '发', '液', 'y', '重', '呜', '条', '丹', '脑', '布', '枯', '走', '上', '疯', '星', '朋', '队', '逃', '妙', '爷', '砖', '管', '能', '欠', '属', '试', '长', '龙', '节', '依', '容', '跟', '板', '穿', '心', '愿', '鼻', '爱', '也', '许', '粥', '鹰', '朽', '编', '值', '飘', '词', '林', ' ', '惚', '反', '使', '病', '怀', '会', '桑', '恼', '静', '沟', '泥', '蔓',

In [7]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]  # 将每个字符转化为索引，得到一个索引的序列
sample = corpus_indices[: 20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [638, 589, 662, 744, 639, 566, 186, 638, 589, 919, 45, 331, 388, 767, 390, 458, 186, 638, 589, 919]


In [9]:
# 随机采样
# 下面的代码每次从数据里随机采样一个小批量。其中批量大小batch_size是每个小批量的样本数，num_steps是每个样本所包含的时间步数。 在随机采样中，每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻。
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n - 1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    example_indices = [i * num_steps for i in range(num_examples)]  # 每个样本的第一个字符在corpus_indices中的下标
    random.shuffle(example_indices)

    def _data(i):
        # 返回从i开始的长为num_steps的序列
        return corpus_indices[i: i + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个随机样本
        batch_indices = example_indices[i: i + batch_size]  # 当前batch的各个样本的首字符的下标
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)

In [10]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[12, 13, 14, 15, 16, 17],
        [ 0,  1,  2,  3,  4,  5]]) 
Y: tensor([[13, 14, 15, 16, 17, 18],
        [ 1,  2,  3,  4,  5,  6]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [18, 19, 20, 21, 22, 23]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [19, 20, 21, 22, 23, 24]]) 



In [27]:
# 相邻采样
# 在相邻采样中，相邻的两个随机小批量在原始序列上的位置相毗邻。
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    print("corpus_len is " + str(corpus_len)) # 30
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    print("corpus_indices is " + str(corpus_indices))
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    print("indices is ")
    print(indices)
    batch_num = (indices.shape[1] - 1) // num_steps
    print("batch_num is " + str(batch_num))
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [28]:
# 同样的设置下，打印相邻采样每次读取的小批量样本的输入X和标签Y。相邻的两个随机小批量在原始序列上的位置相毗邻。
my_seq = list(range(30))
print(my_seq)
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
corpus_len is 30
corpus_indices is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
indices is 
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])
batch_num is 2
X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]]) 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]]) 

